# 🚀 02_08_FuelSystems_Data_Generation.ipynb

This notebook generates **synthetic fuel systems data** for the Databricks AeroDemo pipeline.
It creates component-level metrics focused on the aircraft’s fuel management, efficiency, and system health.

---

### 📋 What this notebook does:

✅ **Sets up aircraft fleet**
- Uses the same fleet IDs as earlier generators (`A320`, `B737`, `A330`)

✅ **Generates daily fuel system records**
- For each aircraft:
  - Simulates **fuel level** and **fuel flow rate**
  - Captures **fuel pressure**, **temperature**, and **tank balance**
  - Includes **pump health** and random anomaly injection (warnings, failures)

✅ **Creates a structured dataset**
- Component records → CSV for Auto Loader (`/Volumes/arao/aerodemo/tmp/fuel`)

✅ **Saves output safely**
- Ensures old sample files are cleaned before saving new ones

---

### 🛠 Key points:
- **Focus area:** Fuel system health, delivery, and balance
- **Data diversity:** Includes normal operations, imbalance alerts, and efficiency metrics
- **Downstream integration:** Feeds DLT tables like `twin_fuel_systems` and predictive analytics models

---

### 🔗 Where this fits:
This notebook is part of the **02_ series** synthetic data generators:
- `02_01_Sensor_Data_Generation.ipynb`
- `02_02_Engine_Data_Generation.ipynb`
- `02_03_CabinPressurization_Data_Generation.ipynb`
- `02_04_Airframe_Synthetic_Data_Generation.ipynb`
- `02_05_LandingGear_Data_Generation.ipynb`
- `02_06_Avionics_Data_Generation.ipynb`
- `02_07_ElectricalSystems_Data_Generation.ipynb`
- `02_08_FuelSystems_Data_Generation.ipynb` (this file)
- Future: additional systems like hydraulics, environmental

In [0]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import random

def generate_fuel_systems_data(num_records_per_aircraft=100):
    aircraft_ids = ["A320_101", "A330_201", "B737_301"]
    data = {
        'fuel_system_id': [],
        'aircraft_id': [],
        'event_timestamp': [],
        'fuel_level': [],
        'fuel_flow_rate': [],
        'fuel_pressure': [],
        'fuel_temperature': [],
        'tank_balance': [],
        'pump_health': [],
        'anomaly_flag': []
    }

    for aircraft_id in aircraft_ids:
        for i in range(num_records_per_aircraft):
            random_days_ago = random.randint(0, 6)
            random_time = datetime.now() - timedelta(
                days=random_days_ago,
                hours=random.randint(0, 23),
                minutes=random.randint(0, 59)
            )
            data['fuel_system_id'].append(f'FS_{aircraft_id}_{i:03d}')
            data['aircraft_id'].append(aircraft_id)
            data['event_timestamp'].append(random_time.strftime("%Y-%m-%d %H:%M:%S"))
            data['fuel_level'].append(round(np.random.uniform(10, 100), 2))  # percentage
            data['fuel_flow_rate'].append(round(np.random.uniform(500, 2000), 2))  # kg/h
            data['fuel_pressure'].append(round(np.random.uniform(20, 50), 2))  # psi
            data['fuel_temperature'].append(round(np.random.uniform(-40, 60), 2))  # °C
            data['tank_balance'].append(round(np.random.uniform(-5, 5), 2))  # kg difference
            data['pump_health'].append(round(np.random.uniform(0.8, 1.0), 2))  # normalized
            data['anomaly_flag'].append(random.choice(['NONE', 'WARN', 'FAIL']))

    df = pd.DataFrame(data)
    return df

# Generate DataFrame
df = generate_fuel_systems_data()

# Save to Auto Loader-compatible path
output_path = "/Volumes/arao/aerodemo/tmp/fuel"
os.makedirs(output_path, exist_ok=True)
output_file = f"{output_path}/fuel_systems_sample.csv"
if os.path.exists(output_file):
    os.remove(output_file)  # safely remove old file

df.to_csv(output_file, index=False)

print(f"✅ Fuel systems data generated: {len(df)} rows saved to {output_file}")